In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import tensorflow as tf
from pandas import read_csv
from numpy import array
from numpy import hstack
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils

from sklearn.metrics import mean_squared_error, r2_score

from pandas import read_csv
from datetime import datetime
from math import sqrt
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Bidirectional
from keras import optimizers

import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import chart_studio.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)

import seaborn as sns    
import matplotlib as mpl
import matplotlib.font_manager as fm

# 데이터 임포트

In [2]:
# 데이터 폴더 경로
path = './차량/'

table = pd.DataFrame()

arrs = os.listdir(path)

fnames = []
for arr in arrs:
    fnames.append(arr)
    
for i in range(len(fnames)): 
    data = pd.read_csv(path+fnames[i], 
                        skiprows=1, 
                        names=['TRANSDATE','ENCACPLATE','INDATE','INGATE','OUTDATE','OUTGATE'])
    table = table.append(data)

    rawdata = table.copy()

In [3]:
rawdata

,TRANSDATE,ENCACPLATE,INDATE,INGATE,OUTDATE,OUTGATE
0,2019-11-20,1BAB337859D379CE3AC47B26F44E328B9D921F42,2019-11-20 21:38:00,북2,2019-11-20 21:40:00,동2
1,2019-11-23,8DF142AD13D70BEC9CC7A3E4A731E806811DEFB5,2019-11-23 17:46:00,동2,2019-11-23 22:29:00,동2
2,2019-11-28,0CBD2C3A5DC0E27AD08283374351889C1082E79B,2019-11-28 08:58:00,서2,2019-11-28 17:08:00,서2
3,2019-11-06,7418F61E7CE72399667F9F1780A9B1EAF8F85B10,2019-11-06 06:16:00,북2,2019-11-06 07:07:00,북2
4,2019-11-03,602B21AF0F85728EFC4739B241FFF25967ED41E3,2019-11-03 15:48:00,동2,2019-11-03 16:23:00,동2
...,...,...,...,...,...,...
173953,2020-10-29,C41183F7EEF04E1B0F07CF9DCB782CDB4AE21429,2020-10-29 16:18:00,동2,2020-10-29 16:23:00,동2
173954,2020-10-29,C652AC6874A52C430497E540A9610327DA18A65B,2020-10-29 09:57:00,북2,2020-10-29 13:01:00,남4
173955,2020-10-29,2CFECAD86A8D9B05CF96081297D9F8DE39841F21,2020-10-29 23:06:00,동2,NaN,NaN
173956,2020-10-29,00436BE3B08712EF3AFDDB3ACF210C3E28C7CC43,2020-10-29 12:10:00,북2,2020-10-29 15:09:00,북2


# 데이터 전처리

In [4]:
# indate가 Nan인 행 제거 
rawdata = rawdata.dropna(subset=['INDATE'])

# outdate가 Nan인 행 제거 
rawdata = rawdata.dropna(subset=['OUTDATE'])
rawdata

# OUTDATE 외의 컬럼 값이 모두 Nan인 행 제거
rawdata = rawdata.dropna(thresh=5)  
rawdata

,TRANSDATE,ENCACPLATE,INDATE,INGATE,OUTDATE,OUTGATE
0,2019-11-20,1BAB337859D379CE3AC47B26F44E328B9D921F42,2019-11-20 21:38:00,북2,2019-11-20 21:40:00,동2
1,2019-11-23,8DF142AD13D70BEC9CC7A3E4A731E806811DEFB5,2019-11-23 17:46:00,동2,2019-11-23 22:29:00,동2
2,2019-11-28,0CBD2C3A5DC0E27AD08283374351889C1082E79B,2019-11-28 08:58:00,서2,2019-11-28 17:08:00,서2
3,2019-11-06,7418F61E7CE72399667F9F1780A9B1EAF8F85B10,2019-11-06 06:16:00,북2,2019-11-06 07:07:00,북2
4,2019-11-03,602B21AF0F85728EFC4739B241FFF25967ED41E3,2019-11-03 15:48:00,동2,2019-11-03 16:23:00,동2
...,...,...,...,...,...,...
173952,2020-10-29,B56123666EA8DC06FB52A4710B678B2041EDD15A,2020-10-29 21:00:00,동2,2020-10-29 22:50:00,동2
173953,2020-10-29,C41183F7EEF04E1B0F07CF9DCB782CDB4AE21429,2020-10-29 16:18:00,동2,2020-10-29 16:23:00,동2
173954,2020-10-29,C652AC6874A52C430497E540A9610327DA18A65B,2020-10-29 09:57:00,북2,2020-10-29 13:01:00,남4
173956,2020-10-29,00436BE3B08712EF3AFDDB3ACF210C3E28C7CC43,2020-10-29 12:10:00,북2,2020-10-29 15:09:00,북2


In [5]:
# 필요한 입차, 출차시간만 남김
df = rawdata[['INDATE','OUTDATE']]
df.reset_index(inplace=True)
del df['index']

#이용객수 카운트를 위한 컬럼 생성
df['cnt'] = 1

#컬럼 INDATE를 날짜, 시간으로 나누기
cols = ['INDATES','INTIME']
df['temp'] = df['INDATE'].str.split(' ')

for i, c in enumerate(cols):
    df[c] = df['temp'].apply(lambda x: x[i])
    
#컬럼 OUTDATE를 날짜, 시간으로 나누기
cols2 = ['OUTDATES','OUTTIME']
df['temp'] = df['OUTDATE'].str.split(' ')

for i, c in enumerate(cols2):
    df[c] = df['temp'].apply(lambda x: x[i])
    
cols3 = ['INYEAR','INMON','INDAY']
cols4 = ['OUTYEAR', 'OUTMON', 'OUTDAY']

#컬럼 INDATES를 날짜, 시간으로 나누기
df['temp'] = df['INDATES'].str.split('-')
for i, c in enumerate(cols3):
    df[c] = df['temp'].apply(lambda x: x[i])
    
#컬럼 OUTDATES를 날짜, 시간으로 나누기
df['temp'] = df['OUTDATES'].str.split('-')
for i, c in enumerate(cols4):
    df[c] = df['temp'].apply(lambda x: x[i])
    
#컬럼 INTIME에서 시간 추출
df['INHOUR'] = df['INTIME'].apply(lambda x: x.split(':')[0])

#컬럼 OUTTIME에서 시간 추출
df['OUTHOUR'] = df['INTIME'].apply(lambda x: x.split(':')[0])

#df에서 필요한 컬럼만 남김
df = df[['INDATES','INYEAR', 'INMON', 'INDAY', 'INHOUR', 'OUTDATES', 'OUTYEAR', 'OUTMON', 'OUTDAY', 'OUTHOUR', 'INDATE', 'OUTDATE', 'cnt']]

In [6]:
df.to_csv('올림픽.csv', encoding='utf-8-sig')

In [7]:
df

,INDATES,INYEAR,INMON,INDAY,INHOUR,OUTDATES,OUTYEAR,OUTMON,OUTDAY,OUTHOUR,INDATE,OUTDATE,cnt
0,2019-11-20,2019,11,20,21,2019-11-20,2019,11,20,21,2019-11-20 21:38:00,2019-11-20 21:40:00,1
1,2019-11-23,2019,11,23,17,2019-11-23,2019,11,23,17,2019-11-23 17:46:00,2019-11-23 22:29:00,1
2,2019-11-28,2019,11,28,08,2019-11-28,2019,11,28,08,2019-11-28 08:58:00,2019-11-28 17:08:00,1
3,2019-11-06,2019,11,06,06,2019-11-06,2019,11,06,06,2019-11-06 06:16:00,2019-11-06 07:07:00,1
4,2019-11-03,2019,11,03,15,2019-11-03,2019,11,03,15,2019-11-03 15:48:00,2019-11-03 16:23:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973670,2020-10-29,2020,10,29,21,2020-10-29,2020,10,29,21,2020-10-29 21:00:00,2020-10-29 22:50:00,1
1973671,2020-10-29,2020,10,29,16,2020-10-29,2020,10,29,16,2020-10-29 16:18:00,2020-10-29 16:23:00,1
1973672,2020-10-29,2020,10,29,09,2020-10-29,2020,10,29,09,2020-10-29 09:57:00,2020-10-29 13:01:00,1
1973673,2020-10-29,2020,10,29,12,2020-10-29,2020,10,29,12,2020-10-29 12:10:00,2020-10-29 15:09:00,1


# 전체 일별 라인 그래프

In [8]:
line = pd.DataFrame(df['cnt'].groupby(df['INDATES']).sum())
line

,cnt
INDATES,
2019-11-01,7791
2019-11-02,8700
2019-11-03,7505
2019-11-04,7578
2019-11-05,7920
...,...
2020-10-27,5150
2020-10-28,5485
2020-10-29,5445


In [9]:
line.iplot(kind='line', title = "올림픽공원 일별 이용객", xTitle='Hour', yTitle = '차량', color='blue') 

# 시간별

In [10]:
hour = pd.DataFrame(df['cnt'].groupby(df['INHOUR']).sum())
hour

,cnt
INHOUR,
00,4995
01,1722
02,1265
03,1378
04,2747
05,16047
06,36192
07,94303
08,248785


In [11]:
hour.iplot(kind='bar', title = "올림픽공원 시간별 차량 이용수", xTitle='Hour', yTitle = '차량', color='skyblue') 

# 월별

In [12]:
mon = pd.DataFrame(df['cnt'].groupby(df['INMON']).sum())
mon

,cnt
INMON,
01,179801
02,150153
03,123714
04,123985
05,291306
06,120720
07,124184
08,121553
09,130397


In [13]:
mon.iplot(kind='bar', title = "올림픽공원 월별 차량 이용수", xTitle='MONTH', yTitle = '차량', color='darkslateblue') 

# 년도별

In [14]:
year = pd.DataFrame(df['cnt'].groupby(df['INYEAR']).sum())
year

,cnt
INYEAR,
2019,439993
2020,1533682


In [15]:
year.iplot(kind='bar', title = "올림픽공원 년도별 차량 이용수", xTitle='YEAR', yTitle = '차량', color='blue') 

# 계절별

In [16]:
sea = df['cnt'].groupby(df['INMON']).sum()

In [17]:
se = ['겨울','봄','여름','가을']

In [18]:
sea = pd.DataFrame(sea)

In [19]:
sea

,cnt
INMON,
01,179801
02,150153
03,123714
04,123985
05,291306
06,120720
07,124184
08,121553
09,130397


In [20]:
sea.reset_index(inplace=True)

In [21]:
win = (sea['cnt'][0] + sea['cnt'][1] + sea['cnt'][11]) / 3
spr = (sea['cnt'][2] + sea['cnt'][3] + sea['cnt'][4]) / 3
summ = (sea['cnt'][5] + sea['cnt'][6] + sea['cnt'][7]) / 3 
aut = (sea['cnt'][8] + sea['cnt'][9] + sea['cnt'][10]) / 3 

In [22]:
season_list = []
season_list.append(win)
season_list.append(spr)
season_list.append(summ)
season_list.append(aut)

In [23]:
table = pd.DataFrame(season_list)
table.index = se

In [24]:
table.iplot(kind='bar', title = "올림픽공원 계절별 차량 이용수", xTitle='Season', yTitle = '차량', color='powderblue') 